# Get gender

## Add `'gender'` key to database

In [2]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
from re import search
from re import UNICODE
import json
import country_converter as coco

In [3]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [4]:
import pandas as pd

In [4]:
UPDATED = True
if UPDATED:
    TMP  = pd.DataFrame(df)
    TMPU = TMP.drop_duplicates('author_id').reset_index(drop=True)
    TMPUI = TMPU[TMPU['first_names'].apply(len)>0].reset_index(drop=True)
    TMPUA = TMPUI[TMPUI['gender_probability']>0.94].reset_index(drop=True)
    
    print(f'TOTAL: {TMP.shape[0]}')
    print(f'Unique autors: {TMPU.shape[0]}')
    print(f'''Remove first names with only intials: {TMPUI.shape[0]} = \
            male:  {TMPU[TMPU['gender']=='male'].shape[0]} + female: {
                    TMPU[TMPU['gender']=='female'].shape[0]} + unknown: {
                    TMPU[TMPU['gender']=='unknown'].shape[0]}.
             ''')
    print(f'Select found gender found probability from 95%: {TMPUA.shape[0]}')
    print(f'female ratio at probability at 95%: { 
                   round(TMPUA[TMPUA['gender']=='female'].shape[0]/(TMPUA[TMPUA['gender']=='female'].shape[0] + TMPUA[TMPUA['gender']=='male'].shape[0]),2)*100  }%')

TOTAL: 57588
Unique autors: 29979
Remove first names with only intials: 20125 =             male:  16013 + female: 3253 + unknown: 10713.
             
Select found gender found probability from 95%: 17634
female ratio at probability at 95%: 16.0%


Fill missing `author_id`s with `unknown` + counter 

In [29]:
DF = pd.DataFrame(df)
DFA = DF[~DF['author_id'].isna()]
DF.shape,DFA.shape

((57588, 15), (57588, 15))

In [30]:
unkown_prefix = 'unknown'
DFB = DF[DF['author_id'].isna()].reset_index(drop=True)
DFB['author_id']=[f'{unkown_prefix}{str(x).zfill(2)}' for x in range(DFB.shape[0])]

In [31]:
TMP = DFB[DFB.duplicated(subset='full_name',keep=False)].sort_values('author_id')

#https://saturncloud.io/blog/how-to-update-a-cell-value-in-pandas-dataframe
i_analysed = []
for i in TMP.index:
    if i not in i_analysed:
        print(i)
        # Extract each set of duplicates
        full_name = TMP.loc[i,'full_name']
        author_id = TMP.loc[i,'author_id']
        i_analysed = i_analysed + list(TMP.loc[TMP['full_name'] == full_name].index)
        DF.loc[ (DF['full_name'] == full_name) & (DF['author_id'].str.contains(unkown_prefix)) , 'author_id'] = author_id

del TMP

In [32]:
DF = pd.concat((DFA,DFB)).reset_index(drop=True)
del DFA
del DFB
DF.shape

(57588, 15)

Prepare names and countries for `gender_guesser` and [https://genderize.io](https://genderize.io) csv input

In [33]:
def remove_initials(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    if (d.get('name') and d.get('name').get('value') ):
        first_names = d.get('name').get('value').split(', ')[-1].replace('-',' ').strip()
    elif d.get('full_name'):
        first_names = d.get('full_name').split(', ')[-1].replace('-',' ').strip()
    else:
        first_names = ''
    x = first_names
    x = sub(r'[\|†•]','',x).strip()
    x = sub(r'\.',' ',   x, UNICODE).strip()
    x = sub(r'\s\w\s','',x, UNICODE).strip()
    x = sub(r'^\w\s','',x, UNICODE).strip()
    x = sub(r'\s\w$','',x, UNICODE).strip()
    x = sub(r'^\w$','',x, UNICODE).strip()
    x = sub(r'^[A-Z]{2}$','',x, UNICODE)
    
    return x

In [34]:
DF['names'] = DF.apply(lambda d: remove_initials(d), axis='columns')

In [35]:
DFU = DF[['country']].drop_duplicates()

In [36]:
DFU['country_id'] = DFU['country'].astype(str).replace('Serbia and Montenegro','Serbia').apply(lambda c: coco.convert(names=c, to='ISO2')).replace('not found','')

None not found in regex


In [37]:
DF = DF.merge(DFU,on='country',how='left')

In [38]:
del DFU

In [39]:
DF['country_id'].unique()[:2]

array(['VE', 'CL'], dtype=object)

In [40]:
DFU = DF[['author_id','names','country','country_id']].rename({'names':'name'},axis='columns').drop_duplicates('author_id').reset_index(drop=True)
DFU.shape

(29979, 4)

`gender_guesser`

In [41]:
D=gender.Detector(case_sensitive=False)
def force_get_gender(d,first_names='names'):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = d.get(first_names)
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [42]:
DFU['gender'] = DFU.apply(lambda d: force_get_gender(d,first_names='name'), axis='columns')

In [43]:
DFU.shape

(29979, 5)

[https://genderize.io](https://genderize.io) csv input

In [44]:
DFUA = DFU[DFU['name'].apply(len)>0].reset_index(drop=True)
DFUB = DFU[DFU['name'].apply(len)==0].reset_index(drop=True)
DFU.shape[0], DFUA.shape[0], DFUB.shape[0],round( DFUB.shape[0] / DFU.shape[0],2)

(29979, 20125, 9854, 0.33)

In [45]:
DFUA.columns

Index(['author_id', 'name', 'country', 'country_id', 'gender'], dtype='object')

In [46]:
DFUA

,author_id,name,country,country_id,gender
0,H.Albrecht.1,Hartwig,Venezuela,VE,male
1,D.M.B.R.1,David Bellorin,Venezuela,VE,male
2,D.F.Mundarain.1,Douglas,Chile,CL,male
3,R.Gaitan.2,Rolando,Venezuela,VE,male
4,J.Ntahompagaze.1,Joseph,Rwanda,RW,male
...,...,...,...,...,...
20120,R.Weber.2,Ronald,Brazil,BR,male
20121,H.J.Drescher.1,Hans Joachim,Germany,DE,male
20122,A.A.P.Videira.1,AntonioP,Brazil,BR,unknown
20123,A.Mocsy.1,Agnes,United States,US,female


In [47]:
DFUA.shape

(20125, 5)

In [48]:
DFUA.to_csv('/home/restrepo/Downloads/genderize.csv',index=False)

Analysis of  [https://genderize.io](https://genderize.io) csv output

In [53]:
csv = 'data/genderize.csv'
DFUA = pd.read_csv(csv)

In [54]:
DFU = pd.concat((DFUA,DFUB)).reset_index(drop=True)

In [55]:
DFU['Gender'] = DFU['Gender'].fillna('unknown')
DFU['Gender Probability'] = DFU['Gender Probability'].fillna(0)
DFU['Gender Count'] = DFU['Gender Count'].fillna(0)
DFU[:1]

,author_id,name,country,country_id,gender,Gender,Gender Probability,Gender Count
0,H.Albrecht.1,Hartwig,Venezuela,VE,male,unknown,0.0,0.0


In [56]:
print('gender-guesser, genderize.io')
DFU[DFU['gender'].apply(lambda s: s in ['male','female'])].shape, DFU[DFU['Gender'].apply(lambda s: s in ['male','female'])].shape

gender-guesser, genderize.io


((16968, 8), (19266, 8))

Update official dataset

In [57]:
d_gender = dict(zip( DFU['author_id'], DFU['Gender'] ) )
d_first_names = dict(zip( DFU['author_id'], DFU.rename({'name':'first_names'},axis='columns')['first_names'] ) )
d_prob   = dict(zip( DFU['author_id'], DFU['Gender Probability'] ) )
unknown = DF[DF['author_id'].str.contains('unknown')]
d_author_id = dict(zip(unknown['full_name'],unknown['author_id']))

In [58]:
UPDATE = False
if UPDATE:
    for author_id in DFU['author_id']:
        kk = [d.update( {'first_names': d_first_names[author_id], 
                         'gender':      d_gender[author_id] ,
                         'gender_probability': d_prob[author_id]  } )
                  for d in df if d.get('author_id') == author_id]

In [6]:
TMP  = pd.DataFrame(df)
TMPU = TMP.drop_duplicates('author_id').reset_index(drop=True)
TMPUI = TMPU[TMPU['first_names'].apply(len)>0].reset_index(drop=True)
TMPUA = TMPUI[TMPUI['gender_probability']>0.9].reset_index(drop=True)

print(f'TOTAL: {TMP.shape[0]}')
print(f'Unique autors: {TMPU.shape[0]}')
print(f'''Remove first names with only intials: {TMPUI.shape[0]} = \
        male:  {TMPU[TMPU['gender']=='male'].shape[0]} + female: {
                TMPU[TMPU['gender']=='female'].shape[0]} + unknown: {
                TMPU[TMPU['gender']=='unknown'].shape[0]}.
         ''')
print(f'Select found gender found probability from 95%: {TMPUA.shape[0]}')
print(f'female ratio at probability at 95%: { 
               round(TMPUA[TMPUA['gender']=='female'].shape[0]/(TMPUA[TMPUA['gender']=='female'].shape[0] + TMPUA[TMPUA['gender']=='male'].shape[0]),2)*100  }%')

TOTAL: 57588
Unique autors: 29979
Remove first names with only intials: 20125 =         male:  16013 + female: 3253 + unknown: 10713.
         
Select found gender found probability from 95%: 17954
female ratio at probability at 95%: 16.0%


In [59]:
TMP  = pd.DataFrame(df)
TMPU = TMP.drop_duplicates('author_id').reset_index(drop=True)
TMPUI = TMPU[TMPU['first_names'].apply(len)>0].reset_index(drop=True)
TMPUA = TMPUI[TMPUI['gender_probability']>0.94].reset_index(drop=True)

print(f'TOTAL: {TMP.shape[0]}')
print(f'Unique autors: {TMPU.shape[0]}')
print(f'''Remove first names with only intials: {TMPUI.shape[0]} = \
        male:  {TMPU[TMPU['gender']=='male'].shape[0]} + female: {
                TMPU[TMPU['gender']=='female'].shape[0]} + unknown: {
                TMPU[TMPU['gender']=='unknown'].shape[0]}.
         ''')
print(f'Select found gender found probability from 95%: {TMPUA.shape[0]}')
print(f'female ratio at probability at 95%: { 
               round(TMPUA[TMPUA['gender']=='female'].shape[0]/(TMPUA[TMPUA['gender']=='female'].shape[0] + TMPUA[TMPUA['gender']=='male'].shape[0]),2)*100  }%')

TOTAL: 57588
Unique autors: 29979
Remove first names with only intials: 20125 =         male:  16013 + female: 3253 + unknown: 10713.
         
Select found gender found probability from 95%: 17634
female ratio at probability at 95%: 16.0%


In [60]:
DUMP = False
if DUMP:
    f=open('data/inspire_LA.json','w')
    json.dump(df,f)
    f.close()

Check test dataset with [https://genderize.io](https://genderize.io)

In [35]:
CO = pd.read_json('https://raw.githubusercontent.com/colav-playground/gender_guesser_first_names/refs/heads/main/data/test_gender.json')
CO.shape

(3967, 3)

In [29]:
UPDATE_CO = False
if UPDATE_CO:
    CO = pd.read_json('https://raw.githubusercontent.com/colav-playground/gender_guesser_first_names/refs/heads/main/data/test_gender.json')
    CO['country_id'] = CO['country'].astype(str).apply(lambda c: coco.convert(names=c, to='ISO2')).replace('not found','')
    CO[CO['country'] == 'other_countries'].to_csv('/home/restrepo/Downloads/test_gender2.csv',index=False)
    CO.to_csv('/home/restrepo/Downloads/test_gender.csv',index=False)

Analysis of  [https://genderize.io](https://genderize.io) test csv output

In [50]:
UPDATE_OC = False
if UPDATE_OC:
    COLD = pd.read_csv('data/test_gender2.csv')    
    CO = pd.read_csv('data/test_gender.csv')
    
    CO = CO[CO['gender']!='L'].reset_index(drop=True)
    
    CO1 = CO[CO['country'] != 'other_countries'].reset_index(drop=True)
    CO2 = CO[CO['country'] == 'other_countries'].reset_index(drop=True)
    CO.shape[0], CO1.shape[0] + CO2.shape[0] 
    
    CO2 = CO2[['names', 'gender', 'country', 'country_id']].merge(
         COLD[['names', 'Gender','Gender Probability', 'Gender Count']],
                on = 'names', how = 'left').fillna('')
    
    CO = pd.concat((CO1,CO2)).reset_index(drop=True)
    CO.to_csv('data/test_gender.csv',index=False)

In [51]:
CO = pd.read_csv('data/test_gender.csv')

In [15]:
COK = CO[ CO['Gender'] != 'unknown'] # male or female

In [16]:
COW = COK[  (COK['gender'] != COK['Gender']) ]
print(f'Number of errors: {COW.shape[0]}')
print( f'Error: {round(COW.shape[0]/COK.shape[0]*100,2)} %. Includes low Gender Probability')

Number of errors: 33
Error: 0.84 %. Includes low Gender Probability


In [19]:
precision = 0.9
good_quality = COK[(COK['Gender Probability'] >= precision)].shape[0]
wrong_good_quality = COK[(COK['Gender Probability'] >= precision) & (COK['gender'] != COK['Gender'])].shape[0]
bad_quality = COK[(COK['Gender Probability'] < precision)].shape[0]
wrong_bad_quality = COK[(COK['Gender Probability'] < precision) & (COK['gender'] != COK['Gender'])].shape[0]

In [20]:
print(f'good_quality_error: {round(wrong_good_quality/good_quality*100,2)} %' )
print(f'bad_quality_error : {round( wrong_bad_quality/ bad_quality*100,2)} %' )

good_quality_error: 0.62 %
bad_quality_error : 19.15 %


with `precision = 0.95`:

good_quality_error: 0.57 %

bad_quality_error : 11.22 %

Conclusion: Is important to guarantee good quality

In [229]:
print(f'female ratio: {round(CO[CO['Gender']=='female'].shape[0]/(CO[CO['Gender']=='female'].shape[0] + CO[CO['Gender']=='male'].shape[0]),2)*100} %')

female ratio: 47.0 %


In [231]:
print(f'unknowns: {CO[ CO['Gender'] == 'unknown'].shape[0]}')
print(f'unknown: {round(CO[ CO['Gender'] == 'unknown'].shape[0]/CO.shape[0]*100,2)} %')

unknowns: 27
unknown: 0.68 %


## Analysis

In [ ]:
import pandas as pd

In [ ]:
DF = pd.read_json('data/inspire_LA.json')

Unique authors ids

In [ ]:
DF.shape[0]

In [ ]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

In [ ]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [ ]:
DF['first_names'] = DF.apply(lambda row: str(row['full_name']).split(', ')[-1]  
                             if row['first_names']=='None' 
                             else row['first_names'] ,axis = 'columns')

In [ ]:
DF['gender'].unique()

In [ ]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

In [ ]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [ ]:
assert male + female + unknown == total

In [ ]:
male+female

In [ ]:
male

In [ ]:
female

In [ ]:
round(female/male,2)

In [ ]:
unknown

In [ ]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

In [ ]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

In [ ]:
DFFM[['author_id','first_names','country','gender']].sample(50)

In [ ]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

check specific entries

In [ ]:
JSON([d for d in df if d.get('author_id')=='F.Fontanot.3'][0])

In [ ]:
import requests
au = requests.get('https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1')
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au['links']['next']

In [ ]:
# https://github.com/inspirehep/rest-api-doc/issues/20#issuecomment-997932079
url='https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1&q=control_number:[1+TO+2852058]'
au = requests.get(url)
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au.status_code

In [ ]:
au = requests.get(url)

In [ ]:
step = 10000
ini = 981872
end = ini + step
for i in range(187):
    print(f'control_number:[{ini} TO {end}]')
    ini = end + 1
    end = ini + step

In [ ]:
au.status_code

In [ ]:
len(au.json()['hits']['hits'])